# import required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# import relevant data from wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/Lagos_State'

html = requests.get(url).content

df_list = pd.read_html(html)

df = df_list[2]

print(df)

df.to_csv('Lagos_data.csv')



            LGA name  Area (km2)  Census 2006 population  \
0              Agege          11                  459939   
1           Alimosho         185                 1277714   
2        Ifako-Ijaye          27                  427878   
3              Ikeja          46                  313196   
4             Kosofe          81                  665393   
5             Mushin          17                  633009   
6       Oshodi-Isolo          45                  621509   
7            Shomolu          12                  402673   
8     Ikeja Division         424                 4801311   
9              Apapa          27                  217362   
10           Eti-Osa         192                  287785   
11      Lagos Island           9                  209437   
12    Lagos Mainland          19                  317720   
13          Surulere          23                  503975   
14    Lagos Division         270                 1542279   
15  Ajeromi-Ifelodun          12        

In [3]:
df_lagpop = pd.read_csv('Lagos_data.csv')
df_lagpop.head()

,Unnamed: 0,LGA name,Area (km2),Census 2006 population,Administrative capital,Postalcode
0,0,Agege,11,459939,Agege,100.0
1,1,Alimosho,185,1277714,Ikotun,100.0
2,2,Ifako-Ijaye,27,427878,Ifako,100.0
3,3,Ikeja,46,313196,Ikeja,100.0
4,4,Kosofe,81,665393,Kosofe,100.0


In [4]:
df_lagpop.columns

Index(['Unnamed: 0', 'LGA name', 'Area (km2)', 'Census 2006 population',
       'Administrative capital', 'Postalcode'],
      dtype='object')

In [5]:
df_lagpop.rename(columns={'LGA name': 'LGA', 'Area (km2)':'landarea', 'Census 2006 population':'population'}, inplace=True)

In [6]:
df_lagpop

,Unnamed: 0,LGA,landarea,population,Administrative capital,Postalcode
0,0,Agege,11,459939,Agege,100.0
1,1,Alimosho,185,1277714,Ikotun,100.0
2,2,Ifako-Ijaye,27,427878,Ifako,100.0
3,3,Ikeja,46,313196,Ikeja,100.0
4,4,Kosofe,81,665393,Kosofe,100.0
5,5,Mushin,17,633009,Mushin,100.0
6,6,Oshodi-Isolo,45,621509,Oshodi/Isolo,100.0
7,7,Shomolu,12,402673,Shomolu,101.0
8,8,Ikeja Division,424,4801311,NaN,NaN
9,9,Apapa,27,217362,Apapa,101.0


In [7]:
df_lag=df_lagpop.drop([8, 14, 19, 21, 24, 25])
df_lag

,Unnamed: 0,LGA,landarea,population,Administrative capital,Postalcode
0,0,Agege,11,459939,Agege,100.0
1,1,Alimosho,185,1277714,Ikotun,100.0
2,2,Ifako-Ijaye,27,427878,Ifako,100.0
3,3,Ikeja,46,313196,Ikeja,100.0
4,4,Kosofe,81,665393,Kosofe,100.0
5,5,Mushin,17,633009,Mushin,100.0
6,6,Oshodi-Isolo,45,621509,Oshodi/Isolo,100.0
7,7,Shomolu,12,402673,Shomolu,101.0
9,9,Apapa,27,217362,Apapa,101.0
10,10,Eti-Osa,192,287785,Ikoyi,101.0


In [8]:
df_lag.reset_index(drop=True, inplace=True)
df_lag

,Unnamed: 0,LGA,landarea,population,Administrative capital,Postalcode
0,0,Agege,11,459939,Agege,100.0
1,1,Alimosho,185,1277714,Ikotun,100.0
2,2,Ifako-Ijaye,27,427878,Ifako,100.0
3,3,Ikeja,46,313196,Ikeja,100.0
4,4,Kosofe,81,665393,Kosofe,100.0
5,5,Mushin,17,633009,Mushin,100.0
6,6,Oshodi-Isolo,45,621509,Oshodi/Isolo,100.0
7,7,Shomolu,12,402673,Shomolu,101.0
8,9,Apapa,27,217362,Apapa,101.0
9,10,Eti-Osa,192,287785,Ikoyi,101.0


In [9]:
df_lag1=df_lag.drop(['Unnamed: 0'], axis = 1)
df_lag1.head(10)

,LGA,landarea,population,Administrative capital,Postalcode
0,Agege,11,459939,Agege,100.0
1,Alimosho,185,1277714,Ikotun,100.0
2,Ifako-Ijaye,27,427878,Ifako,100.0
3,Ikeja,46,313196,Ikeja,100.0
4,Kosofe,81,665393,Kosofe,100.0
5,Mushin,17,633009,Mushin,100.0
6,Oshodi-Isolo,45,621509,Oshodi/Isolo,100.0
7,Shomolu,12,402673,Shomolu,101.0
8,Apapa,27,217362,Apapa,101.0
9,Eti-Osa,192,287785,Ikoyi,101.0


In [10]:
! pip install geocoder

# get coordinate values for the LGA's

In [11]:
import geocoder
latitude=[]
longitude=[]
for code in df_lag1['LGA']:
    g = geocoder.arcgis('{}, Lagos, Nigeria'.format(code))
    print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Lagos, Nigeria'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

Agege [6.6256100000000515, 3.312620000000038]
Alimosho [6.609270000000038, 3.255800000000022]
Ifako-Ijaye [6.651110000000074, 3.3232900000000427]
Ikeja [6.607760000000042, 3.348540000000071]
Kosofe [6.599990000000048, 3.4150900000000206]
Mushin [6.53174000000007, 3.3470100000000684]
Oshodi-Isolo [6.521350000000041, 3.3186300000000415]
Shomolu [6.537850000000049, 3.385340000000042]
Apapa [6.437950000000058, 3.3643600000000333]
Eti-Osa [6.4666800000000535, 3.5832600000000525]
Lagos Island [6.454700000000059, 3.3887600000000475]
Lagos Mainland [6.506430000000023, 3.375530000000026]
Surulere [6.489320000000021, 3.358000000000061]
Ajeromi-Ifelodun [6.459410000000048, 3.3405500000000643]
Amuwo-Odofin [6.445430000000044, 3.2675400000000536]
Ojo [6.462620000000072, 3.166960000000074]
Badagry [6.432160000000067, 2.89265000000006]
Ikorodu [6.6235600000000545, 3.5048300000000268]
Ibeju-Lekki [6.5036700000000565, 3.7330100000000357]
Epe [6.583750000000066, 3.975530000000049]


In [12]:
df2=pd.DataFrame({'Latitude':latitude, 'Longitude':longitude})

df_lag2 = pd.concat([df_lag1 ,df2], axis=1)

df_lag2

,LGA,landarea,population,Administrative capital,Postalcode,Latitude,Longitude
0,Agege,11,459939,Agege,100.0,6.62561,3.31262
1,Alimosho,185,1277714,Ikotun,100.0,6.60927,3.25580
2,Ifako-Ijaye,27,427878,Ifako,100.0,6.65111,3.32329
3,Ikeja,46,313196,Ikeja,100.0,6.60776,3.34854
4,Kosofe,81,665393,Kosofe,100.0,6.59999,3.41509
5,Mushin,17,633009,Mushin,100.0,6.53174,3.34701
6,Oshodi-Isolo,45,621509,Oshodi/Isolo,100.0,6.52135,3.31863
7,Shomolu,12,402673,Shomolu,101.0,6.53785,3.38534
8,Apapa,27,217362,Apapa,101.0,6.43795,3.36436
9,Eti-Osa,192,287785,Ikoyi,101.0,6.46668,3.58326


In [13]:
df_lag2['popdensity']= df_lag2['population']/df_lag2['landarea']
df_lag2

,LGA,landarea,population,Administrative capital,Postalcode,Latitude,Longitude,popdensity
0,Agege,11,459939,Agege,100.0,6.62561,3.31262,41812.636364
1,Alimosho,185,1277714,Ikotun,100.0,6.60927,3.25580,6906.562162
2,Ifako-Ijaye,27,427878,Ifako,100.0,6.65111,3.32329,15847.333333
3,Ikeja,46,313196,Ikeja,100.0,6.60776,3.34854,6808.608696
4,Kosofe,81,665393,Kosofe,100.0,6.59999,3.41509,8214.728395
5,Mushin,17,633009,Mushin,100.0,6.53174,3.34701,37235.823529
6,Oshodi-Isolo,45,621509,Oshodi/Isolo,100.0,6.52135,3.31863,13811.311111
7,Shomolu,12,402673,Shomolu,101.0,6.53785,3.38534,33556.083333
8,Apapa,27,217362,Apapa,101.0,6.43795,3.36436,8050.444444
9,Eti-Osa,192,287785,Ikoyi,101.0,6.46668,3.58326,1498.880208


# show map of lagos with the LGA's

In [14]:
address = 'LAGOS, NIGERIA'

geolocator = Nominatim(user_agent="lag_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lagos are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lagos are 6.4550575, 3.3941795.


In [15]:
map_lagos = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, LGA, popdensity in zip(df_lag2['Latitude'], df_lag2['Longitude'], df_lag2['LGA'], df_lag2['popdensity']):
    label = '{}, {}'.format(LGA, popdensity)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lagos)  
    
map_lagos

# use foursquare location to get food venues in lagos

In [16]:
address = 'LAGOS, NIGERIA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

6.4550575 3.3941795


In [17]:
CLIENT_ID = 'Z2LZWGDYDRPAMB0MQXXDZYJC3Y3O5Z3Y2W1UKGKN1QRVVYWY'
CLIENT_SECRET = 'IQS2SZHXDXRX50YTMPQOK212AD32SHJ3R2CCF3E3OCC1I11H'
ACCESS_TOKEN = 'D3LRK0KWNLUTBOW1VUPZSR4IQ3EOZWWPYYYOKHLPS0KOD0HO'
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z2LZWGDYDRPAMB0MQXXDZYJC3Y3O5Z3Y2W1UKGKN1QRVVYWY
CLIENT_SECRET:IQS2SZHXDXRX50YTMPQOK212AD32SHJ3R2CCF3E3OCC1I11H


In [18]:
search_query = 'food'
radius = 100000
print(search_query + ' .... OK!')

food .... OK!


In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=Z2LZWGDYDRPAMB0MQXXDZYJC3Y3O5Z3Y2W1UKGKN1QRVVYWY&client_secret=IQS2SZHXDXRX50YTMPQOK212AD32SHJ3R2CCF3E3OCC1I11H&ll=6.455,3.394&oauth_token=D3LRK0KWNLUTBOW1VUPZSR4IQ3EOZWWPYYYOKHLPS0KOD0HO&v=20180604&query=food&radius=100000&limit=100'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Z2LZWGDYDRPAMB0MQXXDZYJC3Y3O5Z3Y2W1UKGKN1QRVVYWY&client_secret=IQS2SZHXDXRX50YTMPQOK212AD32SHJ3R2CCF3E3OCC1I11H&ll=6.455,3.394&oauth_token=D3LRK0KWNLUTBOW1VUPZSR4IQ3EOZWWPYYYOKHLPS0KOD0HO&v=20180604&query=food&radius=100000&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fce64789689f0727e3add1d'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '58580c18d25ded6c297a6579',
    'name': 'Food',
    'location': {'address': '217 Etim iyang crescent',
     'crossStreet': 'Muri Okunola Street',
     'lat': 6.433129917612888,
     'lng': 3.435553014278412,
     'labeledLatLngs': [{'label': 'display',
       'lat': 6.433129917612888,
       'lng': 3.435553014278412}],
     'distance': 5201,
     'postalCode': '101241',
     'cc': 'NG',
     'city': 'Victoria Island',
     'state': 'Lagos',
     'country': 'Nigeria',
     'formattedAddress': ['217 Etim iyang crescent (Muri Okunola Street)',
      'Victoria Island 101241',
      'Lagos']},
    'categories': [{'id': '4bf58dd8d48988d1c8941735',
      'name': 'African Restaurant',
      'pluralName': 'African Restaurants',
      'shortName': 'African',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/f

In [21]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,58580c18d25ded6c297a6579,Food,"[{'id': '4bf58dd8d48988d1c8941735', 'name': 'A...",v-1607361656,False,217 Etim iyang crescent,Muri Okunola Street,6.433130,3.435553,"[{'label': 'display', 'lat': 6.433129917612888...",5201,101241,NG,Victoria Island,Lagos,Nigeria,[217 Etim iyang crescent (Muri Okunola Street)...
1,5814ed6238fa8201addc0347,Food Shack,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1607361656,False,NaN,NaN,6.430240,3.416662,"[{'label': 'display', 'lat': 6.4302397, 'lng':...",3725,NaN,NG,Lagos,Lagos,Nigeria,"[Lagos, Lagos]"
2,54c5d6c7498e44dbbda85bdf,Omolgab Foods Nigeria Limited,"[{'id': '52f2ab2ebcbc57f1066b8b3d', 'name': 'P...",v-1607361656,False,"7 Matthew Street off Ojuelegba road, Yaba, Lag...",NaN,6.456851,3.394861,"[{'label': 'display', 'lat': 6.456850710787783...",226,NaN,NG,Lagos,Lagos,Nigeria,"[7 Matthew Street off Ojuelegba road, Yaba, La..."
3,4cebb6388ef78cfafe24ae9b,Leventis Food factory Apapa,[],v-1607361656,False,Dock Yard Road,Apapa,6.448847,3.365529,"[{'label': 'display', 'lat': 6.44884712721841,...",3222,NaN,NG,Lagos,Lagos,Nigeria,"[Dock Yard Road (Apapa), Lagos, Lagos]"
4,532c2270498e0ce7ecc6ae72,Food Spot,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1607361656,False,NaN,NaN,6.429212,3.434986,"[{'label': 'display', 'lat': 6.429212, 'lng': ...",5366,NaN,NG,NaN,NaN,Nigeria,NaN


In [22]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Food,African Restaurant,217 Etim iyang crescent,Muri Okunola Street,6.433130,3.435553,"[{'label': 'display', 'lat': 6.433129917612888...",5201,101241,NG,Victoria Island,Lagos,Nigeria,[217 Etim iyang crescent (Muri Okunola Street)...,58580c18d25ded6c297a6579
1,Food Shack,Fast Food Restaurant,NaN,NaN,6.430240,3.416662,"[{'label': 'display', 'lat': 6.4302397, 'lng':...",3725,NaN,NG,Lagos,Lagos,Nigeria,"[Lagos, Lagos]",5814ed6238fa8201addc0347
2,Omolgab Foods Nigeria Limited,Pop-Up Shop,"7 Matthew Street off Ojuelegba road, Yaba, Lag...",NaN,6.456851,3.394861,"[{'label': 'display', 'lat': 6.456850710787783...",226,NaN,NG,Lagos,Lagos,Nigeria,"[7 Matthew Street off Ojuelegba road, Yaba, La...",54c5d6c7498e44dbbda85bdf
3,Leventis Food factory Apapa,None,Dock Yard Road,Apapa,6.448847,3.365529,"[{'label': 'display', 'lat': 6.44884712721841,...",3222,NaN,NG,Lagos,Lagos,Nigeria,"[Dock Yard Road (Apapa), Lagos, Lagos]",4cebb6388ef78cfafe24ae9b
4,Food Spot,Cafeteria,NaN,NaN,6.429212,3.434986,"[{'label': 'display', 'lat': 6.429212, 'lng': ...",5366,NaN,NG,NaN,NaN,Nigeria,NaN,532c2270498e0ce7ecc6ae72
5,Food step,Food Truck,Ojo Road,NaN,6.463992,3.345732,"[{'label': 'display', 'lat': 6.463991658490377...",5432,NaN,NG,Lagos,Lagos,Nigeria,"[Ojo Road, Lagos, Lagos]",4cce05ca2dc43704402bd208
6,Fresh Food Market,Farmers Market,NaN,NaN,6.433412,3.420211,"[{'label': 'display', 'lat': 6.433411884622203...",3765,NaN,NG,NaN,NaN,Nigeria,NaN,51b22b1d498eff28b3e4c8ac
7,Food O Clock,Restaurant,80 Adeniran Ogunsanya,NaN,6.492529,3.356359,"[{'label': 'display', 'lat': 6.49252932518721,...",5898,NaN,NG,Surulere,Lagos,Nigeria,"[80 Adeniran Ogunsanya, Surulere, Lagos]",560be20b498e8b17febfe8d2
8,F.C.F.M.T Food Canteen,College Cafeteria,"171,Ojo Igbede Road Ajangbadi",NaN,6.420500,3.410216,"[{'label': 'display', 'lat': 6.420499801635742...",4238,NaN,NG,Ojo,NaN,Nigeria,"[171,Ojo Igbede Road Ajangbadi, Ojo]",50969768e4b0a40e1d16f23f
9,Lickies Fast Food,Fast Food Restaurant,Liverpool Road,NaN,6.448038,3.360958,"[{'label': 'display', 'lat': 6.448038, 'lng': ...",3736,NaN,NG,Apapa,Lagos,Nigeria,"[Liverpool Road, Apapa, Lagos]",4c639e6bec94a5931a522cca


In [23]:
print('{} venues were returned by Foursquare.'.format(dataframe_filtered.shape[0]))

50 venues were returned by Foursquare.


### show map of food venues in lagos

In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 


folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Lagos',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


venues_map

# food venues per LGA

In [25]:
CLIENT_ID = 'Z2LZWGDYDRPAMB0MQXXDZYJC3Y3O5Z3Y2W1UKGKN1QRVVYWY' # your Foursquare ID
CLIENT_SECRET = 'IQS2SZHXDXRX50YTMPQOK212AD32SHJ3R2CCF3E3OCC1I11H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z2LZWGDYDRPAMB0MQXXDZYJC3Y3O5Z3Y2W1UKGKN1QRVVYWY
CLIENT_SECRET:IQS2SZHXDXRX50YTMPQOK212AD32SHJ3R2CCF3E3OCC1I11H


In [26]:
df_lag2.loc[0, 'LGA']

'Agege'

In [27]:
LGA_latitude = df_lag2.loc[0, 'Latitude'] # LGA latitude value
LGA_longitude = df_lag2.loc[0, 'Longitude'] # LGA longitude value

LGA_name = df_lag2.loc[0, 'LGA'] # LGA name

print('Latitude and longitude values of {} are {}, {}.'.format(LGA_name, 
                                                               LGA_latitude, 
                                                               LGA_longitude))

Latitude and longitude values of Agege are 6.6256100000000515, 3.312620000000038.


In [28]:
LIMIT = 100

radius = 10000

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query=food&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    LGA_latitude, 
    LGA_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=Z2LZWGDYDRPAMB0MQXXDZYJC3Y3O5Z3Y2W1UKGKN1QRVVYWY&client_secret=IQS2SZHXDXRX50YTMPQOK212AD32SHJ3R2CCF3E3OCC1I11H&v=20180605&ll=6.6256100000000515,3.312620000000038&query=food&radius=10000&limit=100'

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fce6479429b1d3cb581f06c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lagos',
  'headerFullLocation': 'Lagos',
  'headerLocationGranularity': 'city',
  'query': 'food',
  'totalResults': 66,
  'suggestedBounds': {'ne': {'lat': 6.715610090000141,
    'lng': 3.4030560454834977},
   'sw': {'lat': 6.535609909999962, 'lng': 3.2221839545165785}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f75a8e7e4b06c81e0e3ba05',
       'name': 'Wings Restaurant',
       'location': {'lat': 6.576022675981682,
        'lng': 3.3256123316660724,
        'labeledLatLngs': [{'label': 'display',
          'lat': 6.576022675981682,
          'lng': 3.3256123316

In [30]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
foodvenues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
foodvenues =foodvenues.loc[:, filtered_columns]

# filter the category for each row
foodvenues['venue.categories'] = foodvenues.apply(get_category_type, axis=1)

# clean columns
foodvenues.columns = [col.split(".")[-1] for col in foodvenues.columns]

foodvenues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Wings Restaurant,Restaurant,6.576023,3.325612
1,"ICE CREAM PLUS, Medical Road, Ikeja.",Soup Place,6.596598,3.340001
2,Tantalisers,Burger Joint,6.649299,3.265609
3,Tastee Fried Chicken,Fast Food Restaurant,6.631432,3.339814
4,Zen Garden Chinese Resturant,Chinese Restaurant,6.585504,3.357131


In [32]:
def foodvenues(names, latitudes, longitudes, radius=1000):
    
    foodvenues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query=food&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        foodvenues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    foodvenues = pd.DataFrame([item for venue_list in foodvenues_list for item in venue_list])
    foodvenues.columns = ['LGA', 
                  'LGA Latitude', 
                  'LGA Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(foodvenues)

In [33]:
lagosfood_venues = foodvenues(names=df_lag2['LGA'],
                                   latitudes=df_lag2['Latitude'],
                                   longitudes=df_lag2['Longitude']
                                  )

Agege
Alimosho
Ifako-Ijaye
Ikeja
Kosofe
Mushin
Oshodi-Isolo
Shomolu
Apapa
Eti-Osa
Lagos Island
Lagos Mainland
Surulere
Ajeromi-Ifelodun
Amuwo-Odofin
Ojo
Badagry
Ikorodu
Ibeju-Lekki
Epe


In [34]:
print(lagosfood_venues.shape)
lagosfood_venues

(50, 7)


,LGA,LGA Latitude,LGA Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agege,6.62561,3.31262,KFC,6.620788,3.317968,Fast Food Restaurant
1,Ifako-Ijaye,6.65111,3.32329,Savry,6.655729,3.323952,Burger Joint
2,Ifako-Ijaye,6.65111,3.32329,Domino’s Pizza,6.649131,3.323076,Pizza Place
3,Ifako-Ijaye,6.65111,3.32329,Sweet Mother Fast Food,6.656130,3.327601,African Restaurant
4,Ifako-Ijaye,6.65111,3.32329,Downtown Shawarma,6.649090,3.332091,BBQ Joint
5,Ifako-Ijaye,6.65111,3.32329,Mr Biggs,6.649090,3.332091,Burger Joint
6,Ikeja,6.60776,3.34854,University of Suya,6.606512,3.349370,BBQ Joint
7,Ikeja,6.60776,3.34854,Sunrise Chinese Resturant,6.610176,3.345717,Chinese Restaurant
8,Ikeja,6.60776,3.34854,Goat Hunters,6.601134,3.351368,African Restaurant
9,Ikeja,6.60776,3.34854,Tantalizers Allen Junction,6.606133,3.348276,Fast Food Restaurant


In [35]:
lagosfood_onehot = pd.get_dummies(lagosfood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lagosfood_onehot['LGA'] = lagosfood_venues['LGA'] 

# move neighborhood column to the first column
fixed_columns = [lagosfood_onehot.columns[-1]] + list(lagosfood_onehot.columns[:-1])
lagosfood_onehot = lagosfood_onehot[fixed_columns]

lagosfood_onehot.head(10)

,LGA,African Restaurant,BBQ Joint,Bakery,Burger Joint,Burrito Place,Cafeteria,Chinese Restaurant,Diner,Fast Food Restaurant,Food,Food Truck,Fried Chicken Joint,Indian Restaurant,Pizza Place,Restaurant,Soup Place
0,Agege,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,Ifako-Ijaye,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Ifako-Ijaye,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Ifako-Ijaye,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ifako-Ijaye,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Ifako-Ijaye,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,Ikeja,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Ikeja,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
8,Ikeja,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Ikeja,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [36]:
lagosfood_onehot.shape

(50, 17)

In [37]:
lagosfood_grouped = lagosfood_onehot.groupby('LGA').sum().reset_index()
lagosfood_grouped

,LGA,African Restaurant,BBQ Joint,Bakery,Burger Joint,Burrito Place,Cafeteria,Chinese Restaurant,Diner,Fast Food Restaurant,Food,Food Truck,Fried Chicken Joint,Indian Restaurant,Pizza Place,Restaurant,Soup Place
0,Agege,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,Ajeromi-Ifelodun,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Apapa,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0
3,Eti-Osa,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
4,Ifako-Ijaye,1,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0
5,Ikeja,1,1,0,0,0,0,1,0,2,0,0,0,0,0,0,0
6,Ikorodu,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Lagos Island,0,0,0,1,0,0,0,0,4,0,0,1,0,0,0,0
8,Lagos Mainland,1,0,0,0,0,1,1,0,2,0,1,0,0,1,0,0
9,Mushin,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2


# foodvenues clustering using LGA's

In [38]:
kclusters = 2

lagosfood_grouped_clustering = lagosfood_grouped.drop('LGA', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lagosfood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 1, 0, 1, 1, 0], dtype=int32)

In [39]:
lagosfood_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

lagosfood_merged = df_lag2

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
lagosfood_merged = lagosfood_merged.join(lagosfood_grouped.set_index('LGA'), on='LGA')

lagosfood_merged

,LGA,landarea,population,Administrative capital,Postalcode,Latitude,Longitude,popdensity,Cluster Labels,African Restaurant,BBQ Joint,Bakery,Burger Joint,Burrito Place,Cafeteria,Chinese Restaurant,Diner,Fast Food Restaurant,Food,Food Truck,Fried Chicken Joint,Indian Restaurant,Pizza Place,Restaurant,Soup Place
0,Agege,11,459939,Agege,100.0,6.62561,3.31262,41812.636364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alimosho,185,1277714,Ikotun,100.0,6.60927,3.25580,6906.562162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ifako-Ijaye,27,427878,Ifako,100.0,6.65111,3.32329,15847.333333,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Ikeja,46,313196,Ikeja,100.0,6.60776,3.34854,6808.608696,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Kosofe,81,665393,Kosofe,100.0,6.59999,3.41509,8214.728395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Mushin,17,633009,Mushin,100.0,6.53174,3.34701,37235.823529,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
6,Oshodi-Isolo,45,621509,Oshodi/Isolo,100.0,6.52135,3.31863,13811.311111,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Shomolu,12,402673,Shomolu,101.0,6.53785,3.38534,33556.083333,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Apapa,27,217362,Apapa,101.0,6.43795,3.36436,8050.444444,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,Eti-Osa,192,287785,Ikoyi,101.0,6.46668,3.58326,1498.880208,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
lag_food = lagosfood_merged.dropna().reset_index()
lag_food

,index,LGA,landarea,population,Administrative capital,Postalcode,Latitude,Longitude,popdensity,Cluster Labels,African Restaurant,BBQ Joint,Bakery,Burger Joint,Burrito Place,Cafeteria,Chinese Restaurant,Diner,Fast Food Restaurant,Food,Food Truck,Fried Chicken Joint,Indian Restaurant,Pizza Place,Restaurant,Soup Place
0,0,Agege,11,459939,Agege,100.0,6.62561,3.31262,41812.636364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Ifako-Ijaye,27,427878,Ifako,100.0,6.65111,3.32329,15847.333333,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,Ikeja,46,313196,Ikeja,100.0,6.60776,3.34854,6808.608696,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,Mushin,17,633009,Mushin,100.0,6.53174,3.34701,37235.823529,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,6,Oshodi-Isolo,45,621509,Oshodi/Isolo,100.0,6.52135,3.31863,13811.311111,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,7,Shomolu,12,402673,Shomolu,101.0,6.53785,3.38534,33556.083333,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,8,Apapa,27,217362,Apapa,101.0,6.43795,3.36436,8050.444444,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,9,Eti-Osa,192,287785,Ikoyi,101.0,6.46668,3.58326,1498.880208,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,10,Lagos Island,9,209437,Lagos Island,101.0,6.45470,3.38876,23270.777778,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,11,Lagos Mainland,19,317720,Lagos Mainland,101.0,6.50643,3.37553,16722.105263,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [41]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(lag_food['Latitude'], lag_food['Longitude'], lag_food['LGA'], lag_food['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### examination of each clusters

In [42]:
lagosfood_merged.loc[lagosfood_merged['Cluster Labels'] == 0, lagosfood_merged.columns[[1] + list(range(4, lagosfood_merged.shape[1]))]]

,landarea,Postalcode,Latitude,Longitude,popdensity,Cluster Labels,African Restaurant,BBQ Joint,Bakery,Burger Joint,Burrito Place,Cafeteria,Chinese Restaurant,Diner,Fast Food Restaurant,Food,Food Truck,Fried Chicken Joint,Indian Restaurant,Pizza Place,Restaurant,Soup Place
0,11,100.0,6.62561,3.31262,41812.636364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,27,100.0,6.65111,3.32329,15847.333333,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,17,100.0,6.53174,3.34701,37235.823529,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
6,45,100.0,6.52135,3.31863,13811.311111,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,27,101.0,6.43795,3.36436,8050.444444,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
13,12,102.0,6.45941,3.34055,57008.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
15,158,102.0,6.46262,3.16696,3785.259494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17,394,104.0,6.62356,3.50483,1359.439086,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
lagosfood_merged.loc[lagosfood_merged['Cluster Labels'] == 1, lagosfood_merged.columns[[1] + list(range(4, lagosfood_merged.shape[1]))]]

,landarea,Postalcode,Latitude,Longitude,popdensity,Cluster Labels,African Restaurant,BBQ Joint,Bakery,Burger Joint,Burrito Place,Cafeteria,Chinese Restaurant,Diner,Fast Food Restaurant,Food,Food Truck,Fried Chicken Joint,Indian Restaurant,Pizza Place,Restaurant,Soup Place
3,46,100.0,6.60776,3.34854,6808.608696,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,12,101.0,6.53785,3.38534,33556.083333,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,192,101.0,6.46668,3.58326,1498.880208,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,9,101.0,6.45470,3.38876,23270.777778,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11,19,101.0,6.50643,3.37553,16722.105263,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
12,23,101.0,6.48932,3.35800,21911.956522,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0


# number of food venues in each LGA

In [54]:
sort_venues = lagosfood_venues.groupby('LGA').count()
sort_venues

,LGA Latitude,LGA Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
LGA,,,,,,
Agege,1,1,1,1,1,1
Ajeromi-Ifelodun,1,1,1,1,1,1
Apapa,4,4,4,4,4,4
Eti-Osa,2,2,2,2,2,2
Ifako-Ijaye,5,5,5,5,5,5
Ikeja,5,5,5,5,5,5
Ikorodu,1,1,1,1,1,1
Lagos Island,6,6,6,6,6,6
Lagos Mainland,7,7,7,7,7,7


In [55]:
sorted_venues = sort_venues.sort_values('Venue', ascending=False)
sorted_venues

,LGA Latitude,LGA Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
LGA,,,,,,
Lagos Mainland,7,7,7,7,7,7
Surulere,7,7,7,7,7,7
Lagos Island,6,6,6,6,6,6
Ifako-Ijaye,5,5,5,5,5,5
Ikeja,5,5,5,5,5,5
Apapa,4,4,4,4,4,4
Mushin,4,4,4,4,4,4
Oshodi-Isolo,3,3,3,3,3,3
Shomolu,3,3,3,3,3,3
